In [1]:
import numpy as np

import reactord as rd

## Mixture objects

To generate a mixture, first it is necessary to create the Substance objects that will be in the mixture. For instance:

In [13]:
so2 = rd.Substance.from_thermo_database("SO2","SO2")
o2 = rd.Substance.from_thermo_database("oxygen", "O2")
so3 = rd.Substance.from_thermo_database("SO3", "SO3")
co2 = rd.Substance.from_thermo_database("CO2", "CO2")

print(so2.name)
print(o2.name)


SO2
oxygen


A mixture is formed by introducing the objects as a list. In this case, an instance of the IdealGas class is created:

In [14]:
gas_mixture = rd.mix.IdealGas([so2, o2, so3, co2])

# The mixture is a list of substances to which we can access:
print(gas_mixture.substances[0].normal_boiling_point)
print(gas_mixture.substances[1].name)
print(gas_mixture.substances[2].critical_pressure)

263.1
oxygen
8207325.0


Based on the number of moles of each substance in the mixture, molar fractions can be calculated. Furthermore, in order to determine the concentrations (**mol/m³**), temperature (**K**) and pressure (**Pa**) are necessary:

In [16]:
moles = [1.5, 2, 0.5, 3]

molar_fractions = np.round(gas_mixture.mole_fractions(moles), 4)
print("molar fractions: ", molar_fractions, sum(molar_fractions) == 1)

temperature = 298.15  # K
pressure = 101325  # Pa
concentrations = np.round(
    gas_mixture.concentrations(moles, temperature, pressure), 2
)
print("Concentrations (mol/m³): ", concentrations)

molar fractions:  [0.2143 0.2857 0.0714 0.4286] True
Concentrations (mol/m³):  [ 61.31  81.75  20.44 122.62]


The IdealGas class also includes methods to:

-  Calculate the molar volume (mol/m³)
-  Calculate the heat capacity (J/K)
-  Establish the standard formation enthalpies of the substances in the mixture (J/(mol.K)
-  Calculate a term to correct the formation enthalpies at non-standard conditions (J/(mol.K)

In [20]:
moles = [0.5, 0.2, 0.1, 0.2]
temperature = 298.15  # K
pressure = 101325  # Pa

print(
    "Molar volume of the mixture: ",
    np.round(gas_mixture.volume(moles, 273, pressure), 1),
    "m³/mol",
)
print(
    "Heat capacity: ",
    np.round(gas_mixture.mix_heat_capacity(moles, temperature, pressure), 1),
    " J/K",
)

standard_enthalpies = gas_mixture.get_formation_enthalpies()
print("Formation enthalpies: ", standard_enthalpies, " J/(mol.K)")

enthalpy_correction = gas_mixture.formation_enthalpies_correction(400, pressure)
print("Enthalpy correction: ", np.round(enthalpy_correction, 1), " J/(mol.K)")
print(
    "Corrected enthalpies: ",
    np.round(standard_enthalpies + enthalpy_correction, 1),
    " J/(mol.K)",
)

Molar volume of the mixture:  0.0 m³/mol
Heat capacity:  38.4  J/K
Formation enthalpies:  [-296800.       0. -395700. -393474.]  J/(mol.K)
Enthalpy correction:  [4273.3 3025.5 5561.8 4004.7]  J/(mol.K)
Corrected enthalpies:  [-292526.7    3025.5 -390138.2 -389469.3]  J/(mol.K)


A method for calculating the partial pressures is defined within the abstract class AbstractMix:

In [28]:
moles = [8, 20, 15, 12]
mole_fractions = gas_mixture.mole_fractions(moles)
temperature = 700  # K
pressure = 500000  # Pa

partial_pressures = gas_mixture.partial_pressures(mole_fractions, temperature, pressure)
print(np.round(partial_pressures, 0), "Pa")

# The sum of all the partial pressures gives the total pressure:
print(np.sum(partial_pressures), pressure == np.sum(partial_pressures) )

[ 72727. 181818. 136364. 109091.] Pa
500000.0 True
